In [3]:
from Physics import *
from Tools import *
import numpy as np

In [4]:
class DynamicalSimulator(StaticSimulator):
    def __init__(self,Particles,ForceFieldGenerator):
        super().__init__(Particles,ForceFieldGenerator)
        self.Scale  = 1.0
        self.Factor = 10.
        self.Frozen = []
        
    def SimulateEuler(self,Time,Steps):
        t = 0
        dt = Time/Steps
        
        counter = 0
        while t < Time: 
            counter += 1
            t += dt
            self.forceField.Evolve(dt,"Euler")

            for Particle in self.Particles:
                Particle.Acc = self.forceField(Particle.Pos)
                Particle.Evolve(dt)
            
            if counter%100 != 0:
                continue
            
            ToRemove = []
            for i,particle in enumerate(self.Particles):
                if(np.sqrt(particle.Pos*particle.Pos)) > self.Scale*self.Factor:
                    #print("Warning Particle",particle.Nam,"is out of Scope")
                    #print("Freezing it")
                    ToRemove.append(i)
            ToRemove.sort(key=lambda x : -x)
            N = len(self.Particles)
            for i in range(len(ToRemove)):
                    self.Frozen.append(self.Particles[ToRemove[i]])
                    del self.Particles[ToRemove[i]]

In [9]:
Sun1 = Particle("Sun 1",6.0,Vector(5.0/6.0,0,0), Vector(0,2*np.arccos(-1.),0),RED)
Sun2 = Particle("Sun 2",2.0,Vector(-15.0/6.0,0,0),Vector(0,0,0),BLUE)

BinaryStarPotential  = ForceFieldGenerator([Sun1,Sun2])
    
Particles = []
for i in range(2):
    Position = Vector(Random(1,2),0,0)
    Velocity = Vector(0,Random(-12,-10),0)
    Mass = 1E-6   # Irrelevant unless interactions are turned-on        
    Particles.append(Particle("Planet "+str(i+1),Mass,Position,Velocity,WHITE))

In [ ]:
Test_Static = True
Test_Vanilla = False
Test_OrbitFinder = False

Time  = 1
Steps = 2000

BinaryStarPotential = ForceFieldGenerator([Sun1,Sun2])

if Test_Static:

    BinaryStarSimulation = StaticSimulator(Particles,BinaryStarPotential)
    BinaryStarSimulation.ClearTrajectories()
    BinaryStarSimulation.GoToCM()
    BinaryStarSimulation.Simulate(Time,Steps,"Euler")
    BinaryStarSimulation.GeneratePlots(mode="Show")
    #BinaryStarSimulation.GenerateVideo("Binary System 2")
    
if Test_Vanilla:

    BasicSimulator = Simulator([Sun1,Sun2])
    BasicSimulator.ClearTrajectories()
    BasicSimulator.GoToCM()
    BasicSimulator.Simulate(Time,Steps)
    BasicSimulator.GeneratePlots(mode="Show")
    #BasicSimulator.GenerateVideo("Binary System 1")

if Test_OrbitFinder:

    OF = OrbitFinder([],BinaryStarPotential)
    OF.GoToCM()
    OF.FindScale(5,5E-3)

In [18]:
# Here we perform the data collection on the Designed Binary System
# Search Parameters 

BinaryStarPotential = ForceFieldGenerator([Sun1,Sun2])

MinR = 1
MaxR = 15
RSteps = 2
ThetaSteps = 30

Distances = [ MinR + (MaxR-MinR)*i/RSteps for i in range(RSteps+1) ]
Angles    = [ -np.pi + 2*np.pi*i/ThetaSteps for i in range(ThetaSteps+1) ]

M = 0
f = open("Saved Particles.py","w")
for particle in BinaryStarPotential.Particles:
    M += particle.Mas
f.write("Particles = {")
for R in Distances:
    # Here we sample the ring of radius = distance
    Planets = []
    # The speed will be computed according to Kepler's Law
    # v^2 = G(MSuns)/R^3
    T = np.sqrt(4*np.pi**2*R**3/(G*M))
    V = 2*np.pi*R/T
    for i,angle in enumerate(Angles):
        Velocity = Vector(-V*np.sin(angle),V*np.cos(angle),0)
        Position = Vector(R*np.cos(angle),R*np.sin(angle),0)
        Mass = 1E-3
        Part = Particle("Planet "+str(i+1),Mass,Position,Velocity,WHITE)
        Planets.append(Part)
    BinaryStarSimulation = DynamicalSimulator(Planets,BinaryStarPotential)
    BinaryStarSimulation.Scale = R
    BinaryStarSimulation.Factor = 20
    Time = 2
    Steps = 100
    BinaryStarSimulation.ClearTrajectories()
    BinaryStarSimulation.GoToCM()
    BinaryStarSimulation.Simulate(Time,Steps,"Euler")
    BinaryStarSimulation.GeneratePlots({
        "mode"    :"Save",
        "filename":"Simulation_R_"+str(R)})
    print("Done with R =",R)
    f.write('\t"R":'+str(R)+":{\n")
    for particle in BinaryStarSimulation.Particles:
        f.write('\t\t{"Position":Vector'+str(particle.IniPos)+',"Velocity":Vector'+str(particle.IniVel)+"}\n")
        print(particle.IniPos,particle.IniVel)
    f.write("}\n"+("" if R==MaxR else ","))
f.write("}\n")

f.close()

Done with R = 1.0
(-0.10452846326765333,-0.9945218953682734,0) (17.674177442226487,-1.857630904015082,0)
(0.10452846326765346,-0.9945218953682733,0) (17.674177442226483,1.8576309040150842,0)
(0.30901699437494745,-0.9510565162951535,0) (16.901731077888286,5.491705327637735,0)
(-0.5000000000000002,0.8660254037844385,0) (-15.390597961942365,-8.885765876316736,0)
(-0.6691306063588586,0.7431448254773939,0) (-13.20682186277676,-11.891475817565338,0)
Done with R = 8.0
(-8.0,-9.797174393178826e-16,0) (7.694682774887159e-16,-6.283185307179586,0)
(-7.8251808058704455,-1.6632935265420745,0) (1.3063476809370103,-6.1458826331836125,0)
(-7.308363661140806,-3.2538931446064034,0) (2.555601699665417,-5.73997539690064,0)
(-6.472135954999579,-4.702282018339786,0) (3.693163660980914,-5.0832036923152595,0)
(-5.353044850870866,-5.945158603819154,0) (4.6693166485461015,-4.204271594458145,0)
(-4.000000000000002,-6.928203230275508,0) (5.441398092702652,-3.1415926535897944,0)
(-2.4721359549995787,-7.60845213036